# Import Required Libraries

1. Crear un .env en la sección de archivos.
2. Subir la carpeta `tramites_db`.

In [ ]:
%pip install chromadb langchain-openai langchain-core langchain-chroma langchain-huggingface python-dotenv

In [ ]:
# Import necessary libraries
import chromadb
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain

# Load environment variables
load_dotenv()

# Setup Environment Variables
Load environment variables and define constants like embedding model ID and prompt templates.

In [ ]:
# Define constants for embedding model ID and prompt templates
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
QUESTION_PROMPT = PromptTemplate.from_template(
    """
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {input}
    Answer:
    """
)

TOP_K = 3

# Configure Embeddings and Vector Store
Instantiate the embedding model and set up ChromaDB as the vector store with the appropriate collection.

In [ ]:
# Instantiate the embeddings object
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL_ID)  # Added embeddings instantiation

# Initialize a ChromaDB client
chroma_client = chromadb.PersistentClient(path="./tramites_db")

# Create a vector store with the embedding function
vectorstore = Chroma(
    client=chroma_client,
    collection_name="tramites",
    embedding_function=embeddings
)

# Initialize Language Model
Create the OpenAI ChatGPT model instance with appropriate parameters.

In [ ]:
# Initialize the language model
model = ChatOpenAI(model="gpt-4o", temperature=0)

# Create RAG Chain
Build the retrieval-augmented generation chain by combining the retriever and question-answering components.

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(k=TOP_K)

# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(model, QUESTION_PROMPT)

# Build the retrieval-augmented generation (RAG) chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Demo and Usage
Demonstrate how to use the RAG system with example questions and interactive input cells.

In [ ]:
# Demo and Usage

# Function to demonstrate the RAG system with example questions
def demo_rag_system(rag_chain):
    example_questions = [
        "What is the process to renew my passport?",
        "How can I apply for a driver's license?",
        "What documents are required for a visa application?"
    ]
    
    for question in example_questions:
        response = rag_chain.invoke({"input": question})
        print(f"Question: {question}")
        print(f"Answer: {response['answer']}\n")

# Run the demo
demo_rag_system(rag_chain)

# Interactive input cell for user questions
def interactive_rag_system(rag_chain):
    while True:
        question = input("Please enter your question (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            break
        
        response = rag_chain.invoke({"input": question})
        print(f"Answer: {response['answer']}")

# Run the interactive system
interactive_rag_system(rag_chain)